In [1]:
## import tf. and stuff. 
import tensorflow as tf

## import data to work on 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
# set parametrs
learning_rate = .00001
epochs = 10
batch_size = 128

test_valid_size = 256

n_classes = 10
dropout = .75


In [3]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'fc1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out1': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'fcb1': tf.Variable(tf.random_normal([1024])),
    'outb1': tf.Variable(tf.random_normal([n_classes]))
}

In [4]:

def conv2d(x, w, b, stride=1):
    x = tf.nn.conv2d(x, w, strides=[1,stride,stride,1], padding="SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)


def max_pool(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding="SAME")


def conv_net(x, weights, biases, dropout):
    
#     28x28x1 -> 14x14x32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = max_pool(conv1, k=2)
    
#     14x14x32 -> 7x7x64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = max_pool(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1,weights['fc1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['fc1']), biases['fcb1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights['out1']), biases['outb1'])
    
    return out


In [5]:
## about tf graph. 

x = tf.placeholder(tf.float32, [None, 28,28,1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [7]:
logits = conv_net(x, weights, biases, keep_prob)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [9]:
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [10]:
init = tf.global_variables_initializer()


In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 - Loss: 41486.2500 Validation Accuracy: 0.152344
Epoch  1, Batch   2 - Loss: 41588.8438 Validation Accuracy: 0.160156
Epoch  1, Batch   3 - Loss: 33428.7109 Validation Accuracy: 0.171875
Epoch  1, Batch   4 - Loss: 26949.8691 Validation Accuracy: 0.179688
Epoch  1, Batch   5 - Loss: 27698.4785 Validation Accuracy: 0.183594
Epoch  1, Batch   6 - Loss: 22881.4297 Validation Accuracy: 0.195312
Epoch  1, Batch   7 - Loss: 23112.0449 Validation Accuracy: 0.203125
Epoch  1, Batch   8 - Loss: 21952.7617 Validation Accuracy: 0.207031
Epoch  1, Batch   9 - Loss: 22410.4902 Validation Accuracy: 0.214844
Epoch  1, Batch  10 - Loss: 19111.1484 Validation Accuracy: 0.214844
Epoch  1, Batch  11 - Loss: 18240.6055 Validation Accuracy: 0.210938
Epoch  1, Batch  12 - Loss: 16612.1289 Validation Accuracy: 0.222656
Epoch  1, Batch  13 - Loss: 17430.6621 Validation Accuracy: 0.226562
Epoch  1, Batch  14 - Loss: 18591.8301 Validation Accuracy: 0.226562
Epoch  1, Batch  15 - Loss: 17290.

In [ ]:
# # Model
# logits = conv_net(x, weights, biases, keep_prob)

# # Define loss and optimizer
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# # Accuracy
# correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# # Initializing the variables
# init = tf.global_variables_initializer()

# # Launch the graph
# with tf.Session() as sess:
#     sess.run(init)

#     for epoch in range(epochs):
#         for batch in range(mnist.train.num_examples//batch_size):
#             batch_x, batch_y = mnist.train.next_batch(batch_size)
#             sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

#             # Calculate batch loss and accuracy
#             loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
#             valid_acc = sess.run(accuracy, feed_dict={
#                 x: mnist.validation.images[:test_valid_size],
#                 y: mnist.validation.labels[:test_valid_size],
#                 keep_prob: 1.})

#             print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
#                 epoch + 1,
#                 batch + 1,
#                 loss,
#                 valid_acc))

#     # Calculate Test Accuracy
#     test_acc = sess.run(accuracy, feed_dict={
#         x: mnist.test.images[:test_valid_size],
#         y: mnist.test.labels[:test_valid_size],
#         keep_prob: 1.})
#     print('Testing Accuracy: {}'.format(test_acc))


Epoch  1, Batch   1 - Loss: 50002.3359 Validation Accuracy: 0.089844
Epoch  1, Batch   2 - Loss: 42552.6602 Validation Accuracy: 0.089844
Epoch  1, Batch   3 - Loss: 37865.3008 Validation Accuracy: 0.078125
Epoch  1, Batch   4 - Loss: 37466.3750 Validation Accuracy: 0.093750
Epoch  1, Batch   5 - Loss: 32100.5312 Validation Accuracy: 0.101562
Epoch  1, Batch   6 - Loss: 27664.3750 Validation Accuracy: 0.101562
Epoch  1, Batch   7 - Loss: 26379.9102 Validation Accuracy: 0.109375
Epoch  1, Batch   8 - Loss: 23585.9883 Validation Accuracy: 0.121094
Epoch  1, Batch   9 - Loss: 25862.3203 Validation Accuracy: 0.113281
Epoch  1, Batch  10 - Loss: 20046.8477 Validation Accuracy: 0.125000
Epoch  1, Batch  11 - Loss: 22054.7109 Validation Accuracy: 0.132812
Epoch  1, Batch  12 - Loss: 23991.1445 Validation Accuracy: 0.128906
Epoch  1, Batch  13 - Loss: 19006.3086 Validation Accuracy: 0.140625
Epoch  1, Batch  14 - Loss: 19846.0977 Validation Accuracy: 0.136719
Epoch  1, Batch  15 - Loss: 19187.